In [1]:
from numpy import *

In [2]:
def loadDataSet(fileName):
    dataMat=[]
    fr=open(fileName)
    for line in fr.readlines():
        curLine=line.strip().split('\t')
        fltLine=map(float,curLine)
        dataMat.append(fltLine)
    return dataMat

In [3]:
def distEclud(vecA,vecB):
    return sqrt(sum(power(vecA-vecB,2)))

In [4]:
def randCent(dataSet,k):#构造簇的中心点
    n=shape(dataSet)[1]
    centroids=mat(zeros((k,n)))
    for j in range(n):
        minJ=min(dataSet[:,j])
        rangeJ=float(max(dataSet[:,j])-minJ)
        centroids[:,j]=minJ+rangeJ*random.rand(k,1)
    return centroids

In [5]:
datMat=mat(loadDataSet('testSet.txt'))

In [6]:
randCent(datMat,2)

matrix([[ 4.80924644, -3.41291183],
        [ 4.38642443, -2.28664451]])

In [7]:
distEclud(datMat[0],datMat[1])

5.184632816681332

In [8]:
def kMeans(dataSet,k,distMeas=distEclud,createCent=randCent):
    m=shape(dataSet)[0]
    clusterAssment=mat(zeros((m,2)))#one column is for the index of the cluster and the second column is to store the error
    centroids=createCent(dataSet,k)
    clusterChanged=True
    while clusterChanged:
        clusterChanged=False
        for i in range(m):#每一条数据
            minDist=inf;minIndex=-1
            for j in range(k):#该条数据与哪个簇最接近
                distJI=distMeas(centroids[j,:],dataSet[i,:])
                if distJI<minDist:
                    minDist=distJI;minIndex=j
            if clusterAssment[i,0]!=minIndex:#如果发生变化
                clusterChanged=True
            clusterAssment[i,:]=minIndex,minDist**2
        print centroids
        for cent in range(k):#更新簇中心
            ptsInClust=dataSet[nonzero(clusterAssment[:,0].A==cent)[0]]
            centroids[cent,:]=mean(ptsInClust,axis=0)
    return centroids,clusterAssment
    

In [9]:
myCentroids,clustAssing=kMeans(datMat,4)

[[-0.51893452 -0.07613946]
 [-1.42409762  2.52611869]
 [ 0.41080311 -0.74569748]
 [-2.3380327   3.35818024]]
[[-3.61853111 -2.81946867]
 [ 1.06671874  3.24734379]
 [ 2.63377259 -1.66601545]
 [-2.84303986  2.97924629]]
[[-3.38237045 -2.9473363 ]
 [ 2.225975    3.17026943]
 [ 2.8692781  -2.54779119]
 [-2.64677572  2.78993217]]
[[-3.38237045 -2.9473363 ]
 [ 2.6265299   3.10868015]
 [ 2.80293085 -2.7315146 ]
 [-2.46154315  2.78737555]]


In [14]:
def biKmeans(dataSet,k,distMeas=distEclud):
    m=shape(dataSet)[0]
    clusterAssment=mat(zeros((m,2)))
    centroid0=mean(dataSet,axis=0).tolist()[0]
    centList=[centroid0]
    for j in range(m):
        clusterAssment[j,1]=distMeas(mat(centroid0),dataSet[j,:])**2
    while len(centList)<k:
        lowestSSE=inf
        for i in range(len(centList)):
            ptsInCurrCluster=dataSet[nonzero(clusterAssment[:,0].A==i)[0],:]
            centroidMat,splitClustAss=kMeans(ptsInCurrCluster,2,distMeas)
            sseSplit=sum(splitClustAss[:,1])
            sseNotSplit=sum(clusterAssment[nonzero(clusterAssment[:,0].A!=i)[0],1])
            print "sseSplit, and notSplit:",sseSplit,sseNotSplit
            if sseSplit+sseNotSplit<lowestSSE:#寻找最小SSE
                bestCentToSplit=i
                bestNewCents=centroidMat
                bestClustAss=splitClustAss.copy()
                lowestSSE=sseSplit+sseNotSplit
        bestClustAss[nonzero(bestClustAss[:,0].A==1)[0],0]=len(centList)
        bestClustAss[nonzero(bestClustAss[:,0].A==0)[0],0]=bestCentToSplit
        print 'the bestCentToSplit is: ',bestCentToSplit
        print 'the len of bestClustAss is:',len(bestClustAss)
        centList[bestCentToSplit]=bestNewCents[0,:]
        centList.append(bestNewCents[1,:])
        clusterAssment[nonzero(clusterAssment[:,0].A==bestCentToSplit)[0],:]==bestClustAss
    return mat(centList),clusterAssment
        

In [15]:
datMat3=mat(loadDataSet('testSet2.txt'))

In [16]:
biKmeans(datMat3,3)

[[ 1.33713864  4.5134537 ]
 [ 1.70470729  2.12928219]]
[[-1.0607555   3.66998431]
 [ 0.53283171 -0.64411132]]
[[-0.56567979  3.45481879]
 [ 0.37575954 -1.69015508]]
[[-0.06953469  3.29844341]
 [-0.32150057 -2.62473743]]
[[-0.00675605  3.22710297]
 [-0.45965615 -2.7782156 ]]
sseSplit, and notSplit: 453.033489581 0.0
the bestCentToSplit is:  0
the len of bestClustAss is: 60
[[ 2.81525456  0.86773386]
 [ 1.79639851 -2.97959567]]
[[-0.00675605  3.22710297]
 [-0.45965615 -2.7782156 ]]
sseSplit, and notSplit: 453.033489581 0.0


ValueError: min() arg is an empty sequence

In [ ]:
def clusterClubs(numClust=5):
    datList=[]
    for line in open('places.txt').readlines():
        lineArr=line.split('\t')
        datList.append([float(lineArr[4]),float(lineArr[3])])
    datMat=mat(datList)
    myCentroids,clustAssing=biKmeans(datMat,numClust,distMeas=distSLC)
    